<a href="https://colab.research.google.com/github/heisarafat/Breast-Cancer/blob/main/Breast_Cancer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import kagglehub
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
import shap
import gradio as gr
import joblib

In [2]:
bc = pd.read_csv('breast-cancer.csv')

In [3]:
bc.head(5)

In [5]:
# See all column names
print(bc.columns)

In [6]:
# Overview
print(bc.shape)

In [7]:
# Check missing values
print(bc.isnull().sum())

In [8]:
# Drop 'id' since it's not useful
bc.drop(columns=['id'], inplace=True)

In [9]:
# Count unique values in each column
print(bc.nunique())

In [10]:
# EDA
# Class distribution
sns.countplot(x='diagnosis', data=bc)
plt.xticks([0,1], ['Benign', 'Malignant'])
plt.title('Class Distribution')
plt.show()

In [14]:
# Statistical Summary
bc.describe().T  # Transpose for readability

In [15]:
# Encode target
bc['diagnosis'] = bc['diagnosis'].map({'M': 1, 'B': 0})

In [16]:
# Correlation with Target
corr_matrix = bc.corr()
target_corr = corr_matrix['diagnosis'].sort_values(ascending=False)
print(target_corr.head(10))  # Top positively correlated
print(target_corr.tail(10))  # Most negative correlated

# The correlation analysis shows that concave points_worst (0.79), perimeter_worst (0.78), and concave points_mean (0.78) have the strongest positive relationships with the diagnosis, indicating they are highly associated with malignancy.
Conversely, features such as smoothness_se (-0.067) and fractal_dimension_mean (-0.013) show very weak or negative correlation, suggesting minimal predictive power for the target variable.

In [17]:
# Visualize Top Features
top_features = target_corr.index[1:6]  # Skip diagnosis itself
for col in top_features:
    plt.figure(figsize=(6,4))
    sns.kdeplot(data=bc, x=col, hue='diagnosis', fill=True)
    plt.title(f'{col} Distribution by Diagnosis')
    plt.show()

In [18]:
# Correlation Heatmap
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, cmap='coolwarm')
plt.title('Feature Correlation Heatmap')
plt.show()

# The heatmap reveals how each feature relates both to other predictors and to the target variable diagnosis. Features such as concave points_worst, perimeter_worst, radius_worst, and area_worst exhibit strong positive correlations with the diagnosis label, meaning higher values of these measurements are strongly associated with malignant tumors. Conversely, features like smoothness_se and fractal_dimension_mean show little to no correlation with the diagnosis, suggesting they contribute less predictive power on their own.

## Feature Engineering

# To prepare the dataset for modeling, the target variable diagnosis was label-encoded, mapping Malignant (M) to 1 and Benign (B) to 0. Features were then examined for correlation with the target, revealing that variables such as concave points_worst, perimeter_worst, and radius_worst had the strongest positive associations, while features like smoothness_se and fractal_dimension_mean showed weak correlations. This informed later feature selection considerations to reduce redundancy from highly intercorrelated predictors. Finally, feature scaling was applied using StandardScaler to normalize the numerical variables, ensuring uniform influence in the training of machine learning models.

In [20]:
# Separate features and target
x = bc.drop(columns=['diagnosis'])
y = bc['diagnosis']  # target variable

In [21]:
# Split data: 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print("Train Shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)

In [22]:
# Scale the Features
scaler = StandardScaler()

# Fit training data
x_train = scaler.fit_transform(x_train)

# fitting the test data
x_test = scaler.transform(x_test)

# Model Building

In [24]:
# initialize Modelling
model = LogisticRegression(random_state=42)

# Train model
model.fit(x_train, y_train)

# prediction
y_pred = model.predict(x_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [25]:
# Train multiple algorithms for selection of which is best
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
  print(classification_report(y_test, y_pred))
  print("="*50)

In [26]:
# Plot a confusion matrices to see false negative made
models = {
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Plot
fig, axes = plt.subplots(1, 5, figsize=(25, 5))

for ax, (name, model) in zip(axes, models.items()):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=ax, cmap=plt.cm.Blues, colorbar=False)
    ax.set_title(name)

plt.tight_layout()
plt.show()

# Several classification algorithms were implemented to predict the breast cancer diagnosis, including Logistic Regression, K-Nearest Neighbors (KNN), Random Forest, Support Vector Machine (SVM) and Gradient Boosting.

The dataset was split into training and testing subsets to ensure fair performance evaluation. Each model was trained on the training set and then used to predict outcomes on the test set. Performance metrics such as accuracy, precision, recall, and F1-score were calculated, with special emphasis on recall for the malignant class to reduce the risk of false negatives.

To visually compare performance, confusion matrices for all models were plotted side-by-side, allowing for quick identification of strengths and weaknesses across algorithms.

In [27]:
# Hyperparameter Tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5)
grid.fit(x_train, y_train)

print("Best Parameters:", grid.best_params_)
best_svm = grid.best_estimator_

# Model Tuning and Results
To enhance model performance, GridSearchCV was applied to tune the SVM hyperparameters. The search explored different values of C, gamma, and kernel types. The optimal parameters found were:

C: 100

Gamma: 0.001

Kernel: RBF

The model was retrained using these parameters and evaluated on the test set.

In [28]:
# Retrain SVM with best parameters
best_svm = SVC(C=100, gamma=0.001, kernel='rbf', probability=True)
best_svm.fit(x_train, y_train)

In [29]:
# Predict on test set
y_pred = best_svm.predict(x_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Performance Metrics
The tuned model achieved:

Accuracy: 98%

Precision (Benign): 97% — 97% of the cases predicted as benign were actually benign.

Recall (Benign): 100% — All benign cases were correctly identified.

Precision (Malignant): 100% — Every malignant prediction was truly malignant.

Recall (Malignant): 95% — The model correctly identified 95% of malignant cases.

This indicates that the model is highly accurate and balanced in identifying both benign and malignant tumors.

In [30]:
# Confusion Matrix Visualization
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=best_svm.classes_, yticklabels=best_svm.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for Tuned SVM')
plt.show()

# Confusion Matrix Insights

The confusion matrix shows:

* 71 benign cases were all correctly classified (no false negatives for benign).

* 43 malignant cases had 2 false negatives, meaning they were misclassified as benign.

While the false negative count is small, in medical diagnosis, even a single false negative can be critical, as it means a malignant case was missed. This highlights a possible area for further improvement, such as experimenting with different kernels, cost-sensitive learning, or ensemble methods.

In [31]:
# Get probability scores for ROC
best_svm = SVC(C=100, gamma=0.001, kernel='rbf', probability=True)
best_svm.fit(x_train, y_train)

# Predict probabilities for the positive class
y_proba= best_svm.predict_proba(x_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
auc_score = roc_auc_score(y_test, y_proba)

# Plot ROC Curve
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.3f}')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random classifier
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Tuned SVM')
plt.legend(loc='lower right')
plt.show()

print("AUC Score:", auc_score)

# To further validate the model, the Receiver Operating Characteristic (ROC) curve and Area Under the Curve (AUC) were computed. The resulting AUC score of 0.997 indicates that the model can distinguish between malignant and benign cases with 99.7% probability, even when the classification threshold is varied. The ROC curve’s close alignment with the top-left corner demonstrates consistently strong sensitivity and specificity across all thresholds, confirming the model’s exceptional generalization capability.

# Conclusion and Recommendations

The tuned Support Vector Machine model demonstrated strong classification performance in distinguishing malignant from benign breast tumors, supported by a high ROC-AUC score and balanced confusion matrix outcomes. The integration of SHAP interpretability confirmed that the model’s most influential features—particularly concave points_mean, concavity_mean, and texture_worst—are consistent with known medical indicators of malignancy.

Given the high accuracy and interpretability, this model shows potential as a decision-support tool for clinicians. However, before deployment in a real-world clinical environment, the following steps are recommended:

External Validation – Test the model on larger, multi-center datasets to confirm its generalizability across different populations and imaging conditions.

Integration with Clinical Workflows – Develop a user-friendly interface that allows radiologists and oncologists to interact with predictions and explanations in real time.

Bias and Error Analysis – Investigate any demographic or technical biases in the dataset to ensure equitable performance across patient groups.

Periodic Retraining – Implement mechanisms to update the model as new medical data becomes available, maintaining relevance and accuracy.

By combining robust predictive accuracy with interpretable outputs, the model offers a promising foundation for improving early detection and reducing diagnostic errors in breast cancer screening. Future work should focus on enhancing dataset diversity, incorporating additional imaging modalities, and evaluating the tool’s impact in prospective clinical trials.

In [ ]:
import joblib
import pandas as pd
import shap

# 1. Save the trained SVM model
# Ensure 'best_svm' is the name of your final trained model variable
joblib.dump(best_svm, 'svm_model.joblib')

# 2. Save the scaler
# The scaler is crucial because new input must be scaled the same way as the training data
# Ensure 'scaler' is the name of your scaler variable
joblib.dump(scaler, 'scaler.joblib')

# 3. Save the feature names for the UI labels
# 'bc' should be your initial dataframe before dropping the target
feature_names = bc.drop(columns=['diagnosis']).columns.tolist()
joblib.dump(feature_names, 'feature_names.joblib')

# 4. Create and save a SHAP data summary for explaining predictions
# We use a small, representative sample of the training data ('x_train') for the SHAP explainer
shap_summary = shap.kmeans(x_train, 10) # Using 10 summary points
joblib.dump(shap_summary, 'shap_data.joblib')

print("Model, scaler, feature names, and SHAP data have been saved successfully!")

# --- 3. DEFINE THE PREDICTION FUNCTION ---

# This function will take user inputs, process them, and return the results
def predict_cancer(*feature_values):
    try:
        # Convert the 30 input values into a NumPy array
        input_data = np.array([float(val) if val is not None else 0 for val in feature_values]).reshape(1, -1)
    except (ValueError, TypeError):
        return "Error", "Invalid input. Please ensure all 30 fields are filled with numbers.", "", ""

    # Scale the input data using the loaded scaler
    scaled_data = scaler.transform(input_data)

    # --- GET PREDICTION AND CONFIDENCE SCORE ---
    prediction_proba = best_svm.predict_proba(scaled_data)[0]
    prediction = best_svm.predict(scaled_data)[0]

    if prediction == 0:
        diagnosis = "Benign"
        confidence_score = f"{prediction_proba[0] * 100:.2f}%"
    else:
        diagnosis = "Malignant"
        confidence_score = f"{prediction_proba[1] * 100:.2f}%"

    # --- GET TOP CONTRIBUTING FEATURES USING SHAP ---
    explainer = shap.KernelExplainer(best_svm.predict_proba, shap_summary)
    shap_values = explainer.shap_values(scaled_data)[1] # Get values for the "Malignant" class

    # Create a DataFrame for easy analysis
    feature_shap_df = pd.DataFrame({
        'feature': feature_names,
        'shap_value': abs(shap_values)
    })

    # Get the top 3 features with the highest impact
    top_features_df = feature_shap_df.sort_values(by='shap_value', ascending=False).head(3)
    top_features = "\n".join(top_features_df['feature'].tolist())

    # --- GENERATE THE EXPLANATORY NOTE ---
    if diagnosis == "Benign":
        note = "The model predicts the tumor is **Benign**. This means it is likely non-cancerous. This prediction is based on the diagnostic features provided."
    else:
        note = "The model predicts the tumor is **Malignant**. This indicates a high likelihood of being cancerous. Please consult a medical professional for confirmation and further steps."

    return diagnosis, confidence_score, top_features, note


# --- 4. CREATE THE GRADIO INTERFACE ---

# Create a list of input components for the 30 features
input_components = [gr.Number(label=name) for name in feature_names]

# Create the user interface
app = gr.Interface(
    fn=predict_cancer,
    inputs=input_components,
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Textbox(label="Confidence Score"),
        gr.Textbox(label="Top 3 Contributing Features"),
        gr.Markdown(label="What This Prediction Means")
    ],
    title="Breast Cancer Diagnosis Predictor",
    description="Enter the 30 diagnostic feature values below to get a prediction from the SVM model.",
    allow_flagging="never"
)

# --- 5. LAUNCH THE APP ---
app.launch(share=True)